In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#import require modules/packages/libraries
import tensorflow as tf 

from tensorflow import keras 
from keras.models import Sequential, Model 
from keras.layers import Dense, BatchNormalization, Activation, Input 
from keras.utils.np_utils import to_categorical 
from keras.callbacks import ReduceLROnPlateau 

In [ ]:
#read the data
trainf=pd.read_csv('/kaggle/input/digit-recognizer/train.csv', header='infer')
testf=pd.read_csv('/kaggle/input/digit-recognizer/test.csv', header='infer')

#Separate X_train (image) and y_train (class label) and get numpy arrays
y_train=trainf["label"].values
X_train=trainf.iloc[:,1:].values

#numpy arrays of test images
X_test=testf.values

#delete dataframe
del trainf, testf

#Check the data type and size
print(type(X_train), X_train.shape) #ndarray of size 42K X 784
print(type(X_test), X_test.shape) #ndarray of size 28K X 784
print(type(y_train), y_train.shape) #ndarray of size 42K

In [ ]:
X_train=(X_train-127.5)/127.5
X_test=(X_test-127.5)/127.5

#One-hot encode class labels
y_train=to_categorical(y_train)

print(type(y_train), y_train.shape) #Notice the output, y_train is now one-hot encoded
print(X_train.dtype, y_train.dtype, X_test.dtype)

In [ ]:
reduceLROnPlateau=ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

In [ ]:
#Create model

inp=Input(shape=(784,)) 

d1=Dense(units=512)(inp)
b1=BatchNormalization()(d1)
a1=Activation('relu')(b1)

d2=Dense(units=256)(a1)
b2=BatchNormalization()(d2)
a2=Activation('relu')(b2)

d3=Dense(units=128)(a2)
b3=BatchNormalization()(d3)
a3=Activation('relu')(b3)

d4=Dense(units=64)(a3)
b4=BatchNormalization()(d4)
a4=Activation('relu')(b4)

d5=Dense(units=32)(a4)
b5=BatchNormalization()(d5)
a5=Activation('relu')(b5)

d6=Dense(units=10)(a5)
b6=BatchNormalization()(d6)
a6=Activation('softmax')(b6)

ffnn=Model(inputs=inp, outputs=a6)
ffnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

ffnn.summary()
tf.keras.utils.plot_model(ffnn)

In [ ]:
#display the model
tf.keras.utils.plot_model(ffnn)

In [ ]:
#fit the model. We are setting number of epochs = 150 and batch_size is 120. validation_split is 0.2 that
#is 20% data will be used for validation
ffnn.fit(x=X_train, y=y_train, epochs=150, callbacks=[reduceLROnPlateau], batch_size=120, validation_split=0.2)

In [ ]:
predictions=ffnn.predict(X_test)
print(predictions.shape)

predictions=np.argmax(predictions,axis=1) 
print(predictions.shape) 


#read sample_submission.csv in the dataframe. The dataframe will have 2 columns ImageId and Label
sub=pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv', header='infer')

sub["Label"]=predictions

sub.to_csv('submission.csv', index=False)
